### Build Architecture2 - 모델 디버그를 위한 shape출력
- 레이어가 많아질 경우 어떻게 할 것인가?
- 모든 레이어를 다 작성할 것인지?
- 디버그를 위한 shape 출력문은 어떻게 만들 것인지?
- 여러개의 레이어를 통과시킨 후 merge할 경우

In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
# 임의의 데이터 생성

input_data = torch.randn((1,3,244,244))

print(input_data.shape)
print(input_data.dtype)

torch.Size([1, 3, 244, 244])
torch.float32


### 리스트 활용
- 리스트(or nn.ModuleList)에 layer를 담아 활용하기
- forward 함수(디버그)
    - for문을 통해 리스트에서 레이어를 하나씩 가져오고 -> 각 레이어마다 shape을 확인

- forward 함수(디버그 완료 후)
    - nn.Sequential

In [3]:
# ModuleList를 활용

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = [nn.Conv2d(3,5,kernel_size=3),
                  nn.ReLU()]
        self.layers2 = [nn.Conv2d(3,7,kernel_size=3),
                   nn.Conv2d(7,5,kernel_size=1),
                   nn.ReLU()]

        # 두 개의 레이어에 각각 통과시킨 후 merge하기
        self.modules1 = nn.ModuleList(self.layers)
        self.modules2 = nn.ModuleList(self.layers2)

        self.seq1 = nn.Sequential(*self.layers)
        self.seq2 = nn.Sequential(*self.layers2)

        self.fc1 = nn.Linear(5*242*242, 512)

    
    def forward(self, x):
        for idx, module1 in enumerate(self.modules1):
            if idx == 0:
                x1 =  module1(x)
            else:
                x1 = module1(x1)
            print('x1.shape :', x1.shape)

        for idx, module2 in enumerate(self.modules2):
            if idx == 0:
                x2 = module2(x)
            else:
                x2 = module2(x2)
            print('x2 :', x2.shape)

        # merge(addtion)
        conv_out = x1 + x2
        print('conv_out :', conv_out.shape)

        fc_input = conv_out.view(conv_out.size(0), -1)
        print('fc_input:', fc_input.shape)
        fc_output = self.fc1(fc_input)
        print(fc_output.shape)

        return fc_output

In [4]:
model = CNN()
model

CNN(
  (modules1): ModuleList(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (modules2): ModuleList(
    (0): Conv2d(3, 7, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(7, 5, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU()
  )
  (seq1): Sequential(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (seq2): Sequential(
    (0): Conv2d(3, 7, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(7, 5, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU()
  )
  (fc1): Linear(in_features=292820, out_features=512, bias=True)
)

In [ ]:
result = model(input_data)
result

### Cat n Dog 데이터셋 실습

In [21]:
import os
import pandas as pd

def make_cnd_dataframe():
    paths = []
    dataset_type = []
    labels = []
    for dirname, _, filenames in os.walk('/content/drive/MyDrive/고모부_머신러닝/dogncat'):
        for filename in filenames:
            
            if '.jpg' in filename:
                filepath = dirname + '/' + filename
                paths.append(filepath)

            if '/training_set' in filepath:
                dataset_type.append('train')

            elif '/test_set' in filepath:
                dataset_type.append('test')

            else:
                dataset_type.append('N/A')

            if 'dogs' in filepath:
                labels.append('DOG')
            
            elif 'cats' in filepath:
                labels.append('CAT')
            
            else:
                labels.append('N/A')
    
    data_df = pd.DataFrame({'Path' : paths, 'Type' : dataset_type, 'Label' : labels})

    return data_df

        

In [25]:
def make_dataframe_catndog():    
    paths = []
    dataset_type = []
    label = []

    for dirname, _, filenames in os.walk('/content/drive/MyDrive/고모부_머신러닝/dogncat'):
        for filename in filenames:
            if '.jpg' in filename:
                file_path = dirname+'/'+filename
                paths.append(file_path)

            if '/training_set' in file_path:
                dataset_type.append('train')

            elif '/test_set' in file_path:
                dataset_type.append('test')

            else:
                dataset_type.append('N/A') 

            if 'dogs' in file_path:
                label.append('DOG')

            elif 'cats' in file_path:
                label.append('CAT')

            else:
                label.append('N/A')
        
    df = pd.DataFrame({'path' : file_path, 'dataset' : dataset_type, 'label' : label})
    return df

In [26]:
cnd_df = make_dataframe_catndog()

cnd_df.head()

,path,dataset,label
0,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
1,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
2,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
3,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
4,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT


In [27]:
train_df = cnd_df[cnd_df['dataset'] == 'train']
test_df = cnd_df[cnd_df['dataset'] == 'test']

train_path = train_df['path'].values
test_path = test_df['path'].values

train_label = train_df['label'].values
test_label = test_df['label'].values

In [44]:
# 데이터셋 만들기

from torch.utils.data import Dataset,DataLoader
import cv2

class CndDataset(Dataset):
    def __init__(self, paths, labels):
        super(CndDataset, self).__init__()
        self.path = paths
        self.label = labels

    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        image = np.asarray(cv2.cvtColor(cv2.imread(self.path[idx]), cv2.COLOR_BGR2RGB), dtype=np.float32).transpose(2,0,1)

        if self.label is not None:
            label = self.label[idx]

        return image, label

        

In [45]:
cnd_dataset = CndDataset(train_path, train_label)
loader = DataLoader(cnd_dataset, batch_size=10, shuffle=True)


In [61]:
class CNN2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = [nn.Conv2d(3,5,kernel_size=3),
                  nn.ReLU()]
        self.layers2 = [nn.Conv2d(3,7,kernel_size=3),
                   nn.Conv2d(7,5,kernel_size=1),
                   nn.ReLU()]

        # 두 개의 레이어에 각각 통과시킨 후 merge하기
        self.modules1 = nn.ModuleList(self.layers)
        self.modules2 = nn.ModuleList(self.layers2)

        self.seq1 = nn.Sequential(*self.layers)
        self.seq2 = nn.Sequential(*self.layers2)

        self.fc1 = nn.Linear(5 * 228 * 306, 512)

    
    def forward(self, x):
        print('input_image_shape :', x.shape)

        for idx, module1 in enumerate(self.modules1):
            if idx == 0:
                x1 =  module1(x)
            else:
                x1 = module1(x1)
            print('x1.shape :', x1.shape)

        for idx, module2 in enumerate(self.modules2):
            if idx == 0:
                x2 = module2(x)
            else:
                x2 = module2(x2)
            print('x2.shape :', x2.shape)

        # merge(addtion)
        conv_out = x1 + x2
        print('conv_out :', conv_out.shape)

        fc_input = conv_out.view(conv_out.size(0), -1)
        print('fc_input:', fc_input.shape)
        fc_output = self.fc1(fc_input)
        print('fc_output :', fc_output.shape)

        # return fc_output

In [62]:
cnd_model = CNN2()
cnd_model

CNN2(
  (modules1): ModuleList(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (modules2): ModuleList(
    (0): Conv2d(3, 7, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(7, 5, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU()
  )
  (seq1): Sequential(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (seq2): Sequential(
    (0): Conv2d(3, 7, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(7, 5, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU()
  )
  (fc1): Linear(in_features=348840, out_features=512, bias=True)
)

In [63]:
image_array, image_label = next(iter(loader))
cnd_model(image_array)

input_image_shape : torch.Size([10, 3, 230, 308])
x1.shape : torch.Size([10, 5, 228, 306])
x1.shape : torch.Size([10, 5, 228, 306])
x2.shape : torch.Size([10, 7, 228, 306])
x2.shape : torch.Size([10, 5, 228, 306])
x2.shape : torch.Size([10, 5, 228, 306])
conv_out : torch.Size([10, 5, 228, 306])
fc_input: torch.Size([10, 348840])
fc_output : torch.Size([10, 512])


### 데이터의 특성을 파악하기

- 각 배치 내에서 모든 과정의 min 값과 max값을 출력

- 각 배치 내에서 모든 과정의 l2 norm을 프린트해보기

In [38]:
layers = [nn.Conv2d(3, 10, [5,5], padding='same')]
layers.append(nn.ReLU())
layers.append(nn.Conv2d(10, 20, [5,5], stride=2))
layers.append(nn.ReLU())


In [39]:
modules = nn.ModuleList(layers)

In [40]:
seq = nn.Sequential(*layers)

In [41]:
input = torch.randn([1,3,224,224])
print(input.shape)
print(input.dtype)
print(type(input))

torch.Size([1, 3, 224, 224])
torch.float32
<class 'torch.Tensor'>


In [42]:
out = seq(input)

In [43]:
out.shape

torch.Size([1, 20, 110, 110])

In [44]:
o = modules[0](input)
o.shape

torch.Size([1, 10, 224, 224])

In [49]:
o = modules[1](input)
o.shape

torch.Size([1, 3, 224, 224])

In [50]:
o = modules[2](0)

TypeError: ignored

In [ ]:
# 디버그하는 방법. list를 돌면서 하나씩 확인

def forward(input):
    x = input
    for module in modules:
        x = module(x)
        print()

In [ ]:
# 디버그가 완료된 후 nn.Sequential을 사용

def forward(input):
    x = input
    for layer in seq:
        x = module(x)
    return x

### ResNet 구현해보기
- 참조  
    https://towardsdev.com/implement-resnet-with-pytorch-a9fb40a77448

In [93]:
class PrintShape(nn.Module):
    def __init__(self):
        super(PrintShape, self).__init__()

    def forward(self, x):
        print(x.shape)
        return x

In [121]:
from torch.nn.modules.batchnorm import BatchNorm2d
import torch.nn.functional as F
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super(ResBlock, self).__init__()
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.shortcut = nn.Sequential(
                # nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1),
                # nn.BatchNorm2d(out_channels)
            )

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        shortcut = self.shortcut(x)
        # print('shortcut :', shortcut.size())
        x = self.bn1(self.conv1(x))
        x = F.ReLU(x)
        # print('conv1 :', x.size())
        x = self.bn2(self.conv2(x))
        x = F.ReLU(x)
        # print('conv2 :', x.size())
        output = shortcut + x
        # print('output :', x.size())
        return output


In [122]:
class ResNet18(nn.Module):
    def __init__(self, in_channels, resblock, outputs=1000):
        super(ResNet18, self).__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3), # 7x7 kernel을 사용하고 output shape을 1/2로 줄이기 위해서는 padding이 필요
            nn.MaxPool2d(3, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.layer1 = nn.Sequential(
            resblock(64, 64, downsample=False),
            resblock(64, 64, downsample=False)
        )

        self.layer2 = nn.Sequential(
            resblock(64, 128, downsample=True),
            resblock(128, 128, downsample=False)
        )

        self.layer3 = nn.Sequential(
            resblock(128, 256, downsample=True),
            resblock(256, 256, downsample=False)
        )
        
        self.layer4 = nn.Sequential(
            resblock(256, 512, downsample=True),
            resblock(512, 512, downsample=False)
        )

        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(512, 10)
        
    def forward(self, x):
        x = self.layer0(x),
        # print(x.size())
        x = self.layer1(x),
        x = self.layer2(x),
        x = self.layer3(x),
        x = self.layer4(x),
        x = self.gap(x),
        output = self.fc(x)
        # print('output :', output.size())

        return x

In [123]:
from torchsummary import summary

resnet18 = ResNet18(3, ResBlock, outputs=1000)
summary(resnet18, input_data)

TypeError: ignored

### 후기

TypeError: conv2d() received an invalid combination of arguments - got (tuple, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:  
이거 왜 자꾸 나오는거냐..